In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive

/content/gdrive


In [ ]:
cd MyDrive

/content/gdrive/MyDrive


In [ ]:
cd Colab datasets

/content/gdrive/MyDrive/Colab datasets


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as mplot

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
# from skmultilearn.problem_transform import LabelPowerset
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score

import math

In [ ]:
Data = pd.read_csv("multilable_dataset.csv")
Data.head()

,commentText,facts,hypocrisy,warning,affiliation,denouncing,humor,positive_tone,hostile_language
0,kind god hates people curses kind people belie...,0,1,0,0,0,0,1,0
1,question comment racist jews smartest group pe...,0,0,0,0,0,0,1,0
2,israel fail nothing new israel rebellious nati...,0,0,0,0,0,0,1,0
3,god damned ignorant little prick doesnt know a...,0,0,0,0,0,0,0,1
4,shit may cooler police brutality people assumi...,0,1,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

trainA, trainB = train_test_split(Data, test_size=0.50, shuffle=True)
train_dataA = trainA['commentText'].values
train_dataB = trainB['commentText'].values
 
labels=['facts', 'hypocrisy', 'warning', 'affiliation', 'denouncing', 'humor', 'positive_tone', 'hostile_language']
YTrainA = trainA[labels]
YTrainB = trainB[labels]


In [ ]:
train, test = train_test_split(trainA, test_size=0.40, shuffle=True)
train_data = train['commentText'].values
test_data = test['commentText'].values
 
labels=['facts', 'hypocrisy', 'warning', 'affiliation', 'denouncing', 'humor', 'positive_tone', 'hostile_language']
YTrain = train[labels]
YTest = test[labels]

In [ ]:
vect = CountVectorizer(min_df=2, ngram_range=(1, 3))
XTrain = vect.fit(train_data).transform(train_data)
XTest = vect.transform(test_data)

In [ ]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("XTrain : \n{}".format(repr(XTrain)))
print("XTest : \n{}".format(repr(XTest)))
print("Features : {}".format(len(vect.get_feature_names())))

Vocabulary size: 51811
XTrain : 
<10297x51811 sparse matrix of type '<class 'numpy.int64'>'
	with 250248 stored elements in Compressed Sparse Row format>
XTest : 
<6865x51811 sparse matrix of type '<class 'numpy.int64'>'
	with 115864 stored elements in Compressed Sparse Row format>
Features : 51811


### OneVsRest

In [ ]:
categories = list(Data.columns.values)
categories = categories[1:]
print(categories)

LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
lst = []
for category in categories:
    # Training logistic regression model on train data
    LogReg_pipeline.fit(XTrain, YTrain[category])
    
    # calculating test accuracy
    YPred = LogReg_pipeline.predict(XTest)

    tn, fp, fn, tp = confusion_matrix(YTest[category], YPred).ravel()
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    accuracy = (tn+tp)/(tn+tp+fp+fn)
    f1_score = (2*recall*precision)/(recall+precision)

    temp = [precision, recall, accuracy, f1_score]
    lst.append(temp)

    print('Test accuracy of {} is {}'.format(category, accuracy_score(YTest[category], YPred)))

print()
for i in range(len(lst)):
  print(categories[i], lst[i])

['facts', 'hypocrisy', 'warning', 'affiliation', 'denouncing', 'humor', 'positive_tone', 'hostile_language']
Test accuracy of facts is 0.9319737800436999
Test accuracy of hypocrisy is 0.8906045156591406
Test accuracy of warning is 0.9229424617625638
Test accuracy of affiliation is 0.9258557902403496
Test accuracy of denouncing is 0.8566642388929352
Test accuracy of humor is 0.8782228696285507
Test accuracy of positive_tone is 0.9149308084486526
Test accuracy of hostile_language is 0.8182083029861617

facts [0.8233695652173914, 0.4297872340425532, 0.9319737800436999, 0.5647716682199441]
hypocrisy [0.8, 0.35546875, 0.8906045156591406, 0.4922244759972954]
warning [0.8390804597701149, 0.3101983002832861, 0.9229424617625638, 0.45294725956566706]
affiliation [0.8051282051282052, 0.25, 0.9258557902403496, 0.38153098420413123]
denouncing [0.733983286908078, 0.39924242424242423, 0.8566642388929352, 0.5171736997055938]
humor [0.8524173027989822, 0.30098831985624436, 0.8782228696285507, 0.4448871

### BinaryRelevance

In [ ]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.3MB/s 


In [ ]:

from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(LogisticRegression())

# train
classifier.fit(XTrain, YTrain)

# predict
predictions = classifier.predict(XTest)
accuracy_score(YTest,predictions)

tn, fp, fn, tp = confusion_matrix(YTest, predictions).ravel()
recall = tp/(tp+fn)
precision = tp/(tp+fp)
f1_score = (2*recall*precision)/(recall+precision)
print("recall: ",recall,"\nprecision: ",precision,"\naccuracy: ",accuracy_score,"\nf1_score: ",f1_score)


0.5299344501092498

### ClassifierChain

In [ ]:

from skmultilearn.problem_transform import ClassifierChain

# with a gaussian naive bayes base classifier
classifier = ClassifierChain(LogisticRegression())

# train
classifier.fit(XTrain, YTrain)

# predict
predictions = classifier.predict(XTest)

accuracy_score(YTest,predictions)

0.5962126729788784

In [ ]:

# with a gaussian naive bayes base classifier
classifier = ClassifierChain(DecisionTreeClassifier())

# train
classifier.fit(XTrain, YTrain)

# predict
predictions = classifier.predict(XTest)

accuracy_score(YTest,predictions)

0.6211216314639476

### LabelPowerset

In [ ]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(DecisionTreeClassifier())
# train
classifier.fit(XTrain, YTrain)
# predict
predictions = classifier.predict(XTest)
# accuracy
print("Accuracy = ",accuracy_score(YTest,predictions))
print("\n")

Accuracy =  0.6773488710852149




In [ ]:
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression(max_iter=250))
# train
classifier.fit(XTrain, YTrain)
# predict
predictions = classifier.predict(XTest)
# accuracy
print("Accuracy = ",accuracy_score(YTest,predictions))
print("\n")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy =  0.7077931536780772


